In [ ]:
!pip install classiq

In [ ]:
from classiq import (
    Model,
    RegisterUserInput,
    construct_grover_model,
    execute,
    set_constraints,
    show,
    synthesize,
)
from classiq.builtin_functions import ArithmeticOracle, GroverOperator, StatePreparation
from classiq.execution import ExecutionDetails

Game Board

||x1|x2|x3|x4|
|---|---|---|---|---|
|y1|2|-|-|-|
|y2|-|-|3|-|
|y3|-|4|-|-|
|y4|-|-|-|1|


Coversion to SAT problem
|x-axis|m1|m2|m3|m4|
|---|---|---|---|---|
|m1|-|x1|x2|x3|
|m2|x4|-|x5|x6|
|m3|x7|x8|-|x9|
|m4|x10|x11|x12|-|

|y-axis|n1|n2|n3|n4|
|---|---|---|---|---|
|n1|-|y1|y2|y3|
|n2|y4|-|y5|y6|
|n3|y7|y8|-|y9|
|n4|y10|y11|y12|-|

CNF

f(x) = (
((x1) ^ (x2) ^ (x3)) and ((x4) ^ (x5) ^ (x6)) and ((x7) ^ (x8) ^ (x9)) and ((x10) ^ (x11) ^ (12)) and 
((x4) ^ (x7) ^ (x10)) and ((x1) ^ (x8) ^ (x11)) and ((x2) ^ (x5) ^ (x12)) and ((x3) ^ (x6) ^ (x9)) 
)


In [ ]:
formula = "( ((x1) ^ (x2) ^ (x3)) and ((x4) ^ (x5) ^ (x6)) and ((x7) ^ (x8) ^ (x9)) and ((x10) ^ (x11) ^ (12)) and ((x4) ^ (x7) ^ (x10)) and ((x1) ^ (x8) ^ (x11)) and ((x2) ^ (x5) ^ (x12)) and ((x3) ^ (x6) ^ (x9)) )"

register_size = RegisterUserInput(size=1)

model = construct_grover_model(
    expression=formula,
    definitions=[
        ("x1", register_size),
        ("x2", register_size),
        ("x3", register_size),
        ("x4", register_size),
        ("x5", register_size),
        ("x6", register_size),
    ],
    uncomputation_method="optimized",  
    num_reps=1,
)

In [ ]:
qprog = synthesize(model)
show(qprog)

In [ ]:
result = execute(qprog).result()
res = result[0].value
registers = ("x1", "x2", "x3","x4", "x5", "x6")
res_by_regs = res.counts_of_multiple_outputs(registers)
most_common_res = max(res_by_regs, key=res_by_regs.get)
dict(zip(registers, most_common_res))